In [1]:
# Editor appearance set up & Load plot & Calculate DGCI

# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Import packages needed
import gc
import pickle
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename
import platform


from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.widgets import RectangleSelector, PolygonSelector
import numpy as np
from skimage import io, draw
import mplcursors
import IPython.display as Disp
from ipywidgets import widgets
import cv2
from osgeo import gdal
import pandas as pd


# OS related settings
if platform.system() == 'Windows':
    # %matplotlib nbagg
    # Sometimes tk/qt will not let cells rerun after an ERROR occurs
    # %matplotlib tk
    %matplotlib qt
elif platform.system() == 'Darwin':
    Tk().withdraw()
    %matplotlib osx
elif platform == 'linux' or platform == 'linux2':
    ;
# This line of "print" must exist right after %matplotlib command, 
# otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')



# Load image and print size & pre-process

# Use skimage to load multi-layer tiff file

image_file = askopenfilename(title='Load image file', initialdir='./output/field_image')
img = io.imread(image_file)
print("Original Image Shape: ", img.shape)

# Load GPS coordinate from file & Calculate pixel location
plot_loc = askopenfilename(title='Load plot location file', initialdir='./output/plot_location')
try:
    with open(plot_loc, 'rb') as f:
        plot_names = np.array(pickle.load(f))
        plot_vertices_gps = pickle.load(f)
except Exception as e:
    showerror(type(e).__name__, str(e))

# Calculating pixel location from GPS coordinate
ds = gdal.Open(image_file)
gt = ds.GetGeoTransform()
a, b, c, d, e, f = gt

plot_vertices = []
for i in range(len(plot_vertices_gps)):
    one_plot_vertices_gps = plot_vertices_gps[i]
    one_plot_vertices = []
    for vertex in one_plot_vertices_gps:
        xgeo, ygeo = vertex
        
        x = (xgeo - a) / b
        y = (ygeo - d) / f

        one_plot_vertices.append([x, y])
    one_plot_vertices = np.array(one_plot_vertices)
    one_plot_vertices = np.round(one_plot_vertices)
    one_plot_vertices = one_plot_vertices.astype(int)
    plot_vertices.append(one_plot_vertices)

print('Plot location loaded')

all_vertices = np.concatenate(plot_vertices, axis=0)
ul = np.amin(all_vertices, axis=0)
br = np.amax(all_vertices, axis=0)


img = img[ul[1]:br[1], ul[0]:br[0], :]
print("Trimmed Image Shape: ", img.shape)

h, w, d = img.shape

# Extract layers from the multilayer tiff file and do some adjustments
if d == 5:
    layer_RGB = img[:, :, 0:3]
    layer_IR = img[:, :, 3]
    layer_mask = img[:, :, 4]
elif d == 4:
    layer_RGB = img[:, :, 0:3]
    layer_mask = img[:, :, 3]
elif d == 2:
    layer_IR = img[:, :, 0]
    layer_mask = img[:, :, 1]

# Set background to black (You can choose background color when exporting orthomosaic from agisoft, 
# the default is white)
img[np.where(layer_mask == 0)] = 0
# layer_RGB[np.where(layer_mask == 0)] = 0
# layer_IR[np.where(layer_mask == 0)] = 0

# Reduce use of memory
gc.collect()

# Play sound when done
def done():
    print('\nDone')

    # Play sound when done
    
    if platform.system() == 'Windows':
        import winsound
        winsound.Beep(500, 1000)
    elif platform.system() == 'Darwin':
        import os
        os.system( "say Done" )
    elif platform == 'linux' or platform == 'linux2':
        import os
        os.system( "say Done" )

done()

Interactive plot activated
Original Image Shape:  (3296, 2086)


NameError: name 'showerror' is not defined